- 딥러닝
  - 지도학습
    - 예측
    - 분류
  - 비지도학습
    - 차원축소
    - 오토인코더
    - 군집

- 텐서플로 : 예측


In [1]:
import tensorflow as tf

In [3]:
from sklearn.model_selection import train_test_split
(x,y),(x_test, y_test) = tf.keras.datasets.california_housing.load_data()
x_train,x_val,y_train,y_val =  train_test_split(x,y,random_state=123)

In [4]:
x_train.shape

(12384, 8)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [16]:
# 모델 구성
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30,activation='relu',input_shape=x_train.shape[1:]),
    tf.keras.layers.Dense(15,activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss='mse',optimizer='adam')
hist = model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20, batch_size=32)
test_loss = model.evaluate(x_test,y_test)
print(f"test loss : {test_loss}")
from sklearn.metrics import r2_score
y_pred = model.predict(x_test)
r2score = r2_score(y_test,y_pred)
print(f"r2score : {r2score}")

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 208425.5625 - val_loss: 204515.4219
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 206026.2656 - val_loss: 203089.2031
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 205982.0625 - val_loss: 199415.5312
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 200745.2344 - val_loss: 192877.8906
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 192080.0781 - val_loss: 183103.3281
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 184932.8438 - val_loss: 170165.8125
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 170406.2500 - val_loss: 155532.9219
Epoch 8/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 153196.6562 - val_loss: 141495.4375
Epoch 9/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 140589.0469 - val_loss: 128975.4062
Epoch 10/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 129192.7656 - val_loss: 118493.5781
Epoch 11/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/st

In [18]:
# 머신러닝 RendomForest
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(x_train,y_train)
model.score(x_test,y_test)
y_pred = model.predict(x_test)
r2score = r2_score(y_test,y_pred)
print(f"r2score : {r2score}")

r2score : 0.8003450750153709


In [20]:
# 파이토치 -- 덴서구조의 자료구조
# gpu(cuda) cpu 셋팅
# 클래스형태
# 데이터셋과 데이터로더( 배치크기만큼 ) --> 상속 클래스
 # getitem, len
# 모델 : 상속을 받아서 클래스
 # forward : 전방계산, init에서 정의한 각 레이어를 결합

# 학습루프 for
 # for  -- epoch
   # for -- batch
     # 옵티마이져 초기화(기울기)
     # 모델에 학습용데이터 넣어서 예측값
     # 손실함수를 정의해서 손실값을 구하고
     # 손실값을 backward (기울기 계산)
     # 옵티마이져를 통해서 기울기를 적용/업데이트


In [27]:
x_train.shape,y_train.shape
torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1).shape

torch.Size([12384, 1])

In [34]:
import torch
import torch.nn as nn
class DatasetCalifornia(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1)
  def __len__(self):
    return len(self.x)
  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]

from torch.utils.data import DataLoader
train_dataset = DatasetCalifornia(x_train, y_train)
val_dataset = DatasetCalifornia(x_val, y_val)
test_dataset = DatasetCalifornia(x_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 모델생성
class CaliforniaModel(nn.Module):
  def __init__(self):
    super(CaliforniaModel,self).__init__()
    self.fc1 = nn.Linear(x_train.shape[1],30)
    self.fc2 = nn.Linear(30,15)
    self.fc3 = nn.Linear(15,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x
model = CaliforniaModel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # 옵티마이져
loss_fn = nn.MSELoss() # 손실함수
# 학습루프
from tqdm import tqdm
epochs = 20
for epoch in range(epochs):
  iterator = tqdm(train_dataloader)
  for x,y in iterator:
    optimizer.zero_grad()  # 기울기 초기화
    y_pred = model(x)  # 예측
    loss = loss_fn(y_pred,y) # 손실값(텐서)
    loss.backward() # 기울기 계산
    optimizer.step() # 기울기 업데이트
    iterator.set_description(f"epoch : {epoch+1} loss : {loss.item()}")
# 모델을 저장
torch.save(model.state_dict(),'california.pth')
# 모델 불러오기
model = CaliforniaModel()
model.load_state_dict(torch.load('california.pth'))
# 모델 평가
test_loss = 0.0
with torch.no_grad():
  for x,y in test_dataloader:
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    test_loss += loss.item()
test_loss /= len(test_dataloader)
print(f"test loss : {test_loss}")
from sklearn.metrics import r2_score
y_pred = model(torch.tensor(x_test, dtype=torch.float32))
r2score = r2_score(y_test,y_pred.detach().numpy() )
print(f"r2score : {r2score}")

epoch : 20 loss : 12378242048.0: 100%|██████████| 387/387 [00:02<00:00, 147.64it/s]
<ipython-input-34-f1f92cb7b67f>:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.l

test loss : 9929519530.666666
r2score : 0.2531021237373352


- 분류
  - 텐서플로
  - 파이토치

In [37]:
# fashionmnist 분류
(x,y),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [38]:
# 텐서플로
x = x / 255.0
x_test = x_test / 255.0

In [39]:
x_train,x_val,y_train,y_val = train_test_split(x,y,stratify=y,random_state=45)

In [40]:
x_train.shape

(45000, 28, 28)

In [41]:
# c-p-c-p-c-flatten-fc1-fc1
tensorflow_model = tf.keras.Sequential([
    # 입력 레이어
    tf.keras.layers.Input(shape=x_train.shape[1:]),
    # 채널정보 추가
    tf.keras.layers.Reshape(target_shape=(28,28,1)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])
tensorflow_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
hist = tensorflow_model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=10,batch_size=32)
test_loss, test_acc = tensorflow_model.evaluate(x_test,y_test)
print(f"test loss : {test_loss}")
print(f"test acc : {test_acc}")
from sklearn.metrics import classification_report
import numpy as np
y_pred = tensorflow_model.predict(x_test)
y_pred = np.argmax(y_pred,axis=1)  # 10개 클래스에 대한 각각의 확률값의 최고값에 해당하는 인덱스가 곧 클래스번호
print(classification_report(y_test,y_pred))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 63s 43ms/step - accuracy: 0.7347 - loss: 0.7171 - val_accuracy: 0.8671 - val_loss: 0.3695
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 79s 41ms/step - accuracy: 0.8723 - loss: 0.3439 - val_accuracy: 0.8904 - val_loss: 0.3036
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 81s 41ms/step - accuracy: 0.8952 - loss: 0.2791 - val_accuracy: 0.8897 - val_loss: 0.2995
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 83s 42ms/step - accuracy: 0.9097 - loss: 0.2408 - val_accuracy: 0.9032 - val_loss: 0.2680
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 57s 40ms/step - accuracy: 0.9182 - loss: 0.2194 - val_accuracy: 0.9081 - val_loss: 0.2537
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 83s 41ms/step - accuracy: 0.9308 - loss: 0.1883 - val_accuracy: 0.9060 - val_loss: 0.2655
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 83s 42ms/step - accuracy: 0.9388 - loss: 0.1635 - val_accuracy: 0.9036 - val_loss: 0.2671
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 80s 40ms/step - accuracy: 0.9468 -

In [42]:
x_train.shape

(45000, 28, 28)

In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# fashionmnist 분류
(x,y),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
# 텐서플로
x = x / 255.0
x_test = x_test / 255.0
x_train,x_val,y_train,y_val = train_test_split(x,y,stratify=y,random_state=45)

class FashionMnistDataset(torch.utils.data.Dataset):
  def __init__(self, x, y):
    self.x = torch.tensor(x, dtype=torch.float32)
    self.x = self.x.unsqueeze(1)  #origin (none, 28 28) (none,1,28,28)
    self.y = torch.tensor(y, dtype=torch.long)
  def __len__(self):
    return len(self.x)
  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]
# 데이터 로더
train_dataset = FashionMnistDataset(x_train, y_train)
val_dataset = FashionMnistDataset(x_val, y_val)
test_dataset = FashionMnistDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 모델 정의
# c-p-c-p-c-flattern-fc-fc
class FashionMnist(nn.Module):
  def __init__(self):
    super(FashionMnist,self).__init__()
    self.conv1 = nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)  # same padding
    self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2) # 14 14
    self.conv2 = nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)  # same padding
    self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2) # 7 7
    self.conv3 = nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1)  # same padding
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(128*7*7,128)
    self.fc2 = nn.Linear(128,10)
  def forward(self,x):
    x = torch.relu( self.conv1(x) )
    x = self.pool1(x)
    x = torch.relu( self.conv2(x) )
    x = self.pool2(x)
    x = self.conv3(x)
    x = self.flatten(x)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = FashionMnist().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # 옵티마이져
loss_fn = nn.CrossEntropyLoss() # 손실함수
# 학습루프
# 파이토치의 conv 는 (batchsize,channel,....)
for epoch in range(10):
  iterator = tqdm(train_loader)
  for x,y in iterator:
    x = x.to(device)
    y= y.to(device)
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    loss.backward()
    optimizer.step()
    iterator.set_description(f"epoch : {epoch+1} loss : {loss.item()}")

# 모델을 저장
torch.save(model.state_dict(),'fashionmnist.pth')
# 모델 불러오기
model = FashionMnist()
model.load_state_dict(torch.load('fashionmnist.pth'))
# 모델 평가
test_loss = 0.0
with torch.no_grad():
  for x,y in test_loader:
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    test_loss += loss.item()
test_loss /= len(test_loader)
print(f"test loss : {test_loss}")

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


epoch : 10 loss : 0.09142746031284332: 100%|██████████| 1407/1407 [00:09<00:00, 154.48it/s]
<ipython-input-1-4c109035e69d>:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  

test loss : 0.33780995050903423


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 10000, 28, 28] to have 1 channels, but got 10000 channels instead

In [19]:
from sklearn.metrics import classification_report
import numpy as np
data,target = next(iter(test_loader))
pred = model(data)  # (32,10)
pred = np.argmax(pred.detach().numpy(),axis=1)
# y_pred = np.argmax(y_pred,axis=1)
print(classification_report(target, pred))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      1.00      1.00         3
           2       0.67      1.00      0.80         4
           3       0.50      1.00      0.67         1
           4       0.80      0.67      0.73         6
           5       1.00      1.00      1.00         4
           6       0.50      0.50      0.50         2
           7       1.00      1.00      1.00         3
           8       1.00      0.67      0.80         3
           9       1.00      1.00      1.00         4

    accuracy                           0.84        32
   macro avg       0.85      0.83      0.82        32
weighted avg       0.87      0.84      0.84        32

